In [1]:
# Set environmental parameter in order to make brian 2 work
# import os
# os.environ['MACOSX_DEPLOYMENT_TARGET'] = '10.9'

In [2]:
# import brian and required packages
from brian2 import *
import math
from IPython.core.debugger import Pdb
ipdb = Pdb()
%matplotlib inline

Current Clamp Mouse L2/3 Barrel cortex Pyramidal Neuron Model.

In [3]:
#passive neuronal parameters
Cm = 0.75*uF/cm**2 #membrane capicitance
Ri = 200*ohm*cm #axial Resistance
EL = -70*mV #leak reversal potential
ENa = 55*mV #sodium reversal potential
EK = -90*mV #potassium reversal potential

#additional parameters
defaultclock.dt = 0.025*ms
duration = 500*ms
param = math.log(10) #used in neuron equations

#neuron morphology
morpho = Soma(50*um)  #Create soma
diameters = hstack([linspace(4, 1, 11), ones(290)])*um #create tapering
morpho.axon = Section(diameter=diameters, length=ones(300)*um, n=300) #create axon

#set neuron equations
eqs='''
Im = gL*(EL - v) + (m**3)*gNa*h*(ENa - v) + (n**4)*gK*(EK - v)  : amp/meter**2

I : amp (point current)

# activation gates Na channel
dm/dt = (minf-m)/taum : 1 
minf = (1 / (1 + exp((v+25.5*mV) / (-5.29*mV)))) : 1
taum = 2.64*ms - (2.52*ms/(1 + exp((v+120*mV) / (-25*mV)))) : second

# inactivation gate Na channel
dh/dt = (hinf-h)/tauh : 1
hinf = (1 / (1 + exp((v+48.9*mV) / (5.18*mV)))) : 1
tauh = (2*ms/(1 + exp((v+62.9*mV) / (-10*mV)))) + (1*ms/(1 + exp((v+34.9*mV) / (3.6*mV)))) : second

# activation gates K channel
dn/dt = alphan * (1-n) - betan * n : 1
alphan =(1 / exp(((param*((-0.021 * v + (1.14*mV))/mV)))))/ms : Hz
betan = (1 / exp(((param*((0.031 * v + (2.8*mV))/mV)))))/ms : Hz

gNa : siemens/metre**2
gK : siemens/metre**2
gL : siemens/metre**2
'''

#create spatial neuron
neuron = SpatialNeuron(morphology=morpho, model=eqs, Cm=Cm, Ri=Ri, threshold='v > -40*mV',
                    refractory='v > -40*mV', method='exponential_euler')

#set neuronal parameters
neuron.gL[0] = 0.5*nS/morpho.area[0] #leak conductance soma
neuron.axon.gL = 0.33*nS/sum(morpho.axon.area) #leak conductance whole axon incl. AIS

neuron.gK[0] = 10*nS/morpho.area[0] #K conductance soma
neuron.axon.gK[10:25] = 10*nS/sum(morpho.axon.area[10:25]) # K conductance AIS

neuron.gNa[0] = 300*nS/morpho.area[0] #NA conductance soma
neuron.axon[10:25].gNa = 3000*nS/sum(morpho.axon.area[10:25]) # NA conductance AIS

neuron.v = EL #membrane voltage
neuron.h = 1 #inactivation gate NA
neuron.m = 0 #activation gate NA
neuron.n = 0.5 #activation gate K

#set current injection into soma
neuron.I[0] = 0*nA

#monitor required variables
spikes = SpikeMonitor(neuron) #record spikes
mon = StateMonitor(neuron, ['v'], record = True) #record additional neuron parameters

#run
run(duration)

Voltage clamp Step and Hold Mouse L2/3 Barrel cortex Pyramidal Neuron Model.

In [4]:
#passive neuronal parameters
Cm = 0.75*uF/cm**2 #membrane Capicitance
Ri = 200*ohm*cm #axial Resistance
EL = -70*mV #leak reversal potential
ENa = 55*mV #sodium reversal potential
EK = -90*mV #potassium reversal potential

#additional parameters
defaultclock.dt = 0.025*ms
duration = 500*ms
param = math.log(10) # used in neuron equations

#neuron morphology
morpho = Soma(50*um)  #create soma
diameters = hstack([linspace(4, 1, 11), ones(290)])*um #create tapering
morpho.axon = Section(diameter=diameters, length=ones(300)*um, n=300) #create Axon

#set neuron equations
eqs='''
Im = gL*(EL - v) + (m**3)*gNa*h*(ENa - v) + (n**4)*gK*(EK - v) + gClamp*(vc - v) : amp/meter**2
Iclamp = gClamp*(vc-v) : amp/meter**2

# activation gates Na channel
dm/dt = (minf-m)/taum : 1 
minf = (1 / (1 + exp((v+25.5*mV) / (-5.29*mV)))) : 1
taum = 2.64*ms - (2.52*ms/(1 + exp((v+120*mV) / (-25*mV)))) : second

# inactivation gate Na channel
dh/dt = (hinf-h)/tauh : 1
hinf = (1 / (1 + exp((v+48.9*mV) / (5.18*mV)))) : 1
tauh = (2*ms/(1 + exp((v+62.9*mV) / (-10*mV)))) + (1*ms/(1 + exp((v+34.9*mV) / (3.6*mV)))) : second

# activation gates K channel
dn/dt = alphan * (1-n) - betan * n : 1
alphan =(1 / exp(((param*((-0.021 * v + (1.14*mV))/mV)))))/ms : Hz
betan = (1 / exp(((param*((0.031 * v + (2.8*mV))/mV)))))/ms : Hz

gNa : siemens/metre**2
gK : siemens/metre**2
gL : siemens/metre**2
gClamp : siemens/metre**2

vc : volt (shared) #step and hold protocol
'''

#create spatial neuron
neuron = SpatialNeuron(morphology=morpho, model=eqs, Cm=Cm, Ri=Ri, threshold='v > -40*mV',
                    refractory='v > -40*mV', method='exponential_euler')

#set neuronal parameters
neuron.gL[0] = 0.5*nS/morpho.area[0] #leak conductance soma
neuron.axon.gL = 0.33*nS/sum(morpho.axon.area) #leak conductance whole axon incl. AIS

neuron.gK[0] = 10*nS/morpho.area[0] #K conductance soma
neuron.axon.gK[10:25] = 10*nS/sum(morpho.axon.area[10:25]) #K conductance AIS

neuron.gNa[0] = 300*nS/morpho.area[0] #NA conductance soma
neuron.axon[10:25].gNa = 3000*nS/sum(morpho.axon.area[10:25]) #NA conductance AIS

neuron.gClamp = (0.5*nS/morpho.area[0])*400 #injected current conductance

neuron.v = EL #membrane voltage
neuron.h = 1 #inactivation gate NA
neuron.m = 0 #activation gate NA
neuron.n = 0.5 #activation gate K

#monitor required variables
spikes = SpikeMonitor(neuron) #record spikes
mon = StateMonitor(neuron, ['Iclamp'], record = True) # record additional neuron parameters

#run
steps = linspace(0, 12, 2) #number of steps
store = np.zeros((len(steps), len(linspace(0, duration*3, (duration*3)/defaultclock.dt)))) #voltage traces per step
k = 0 #track step count
start = 0
for i in steps:
    run(duration)
    neuron.vc = -70*mV + (i*10*mV) #step potential
    run(duration)
    neuron.vc = -70*mV #restpotenital
    run(duration)
    end = len(mon.Iclamp[0]) #determine end of voltage trace of step i
    store[k] = mon.Iclamp[0][start:end]*neuron.area[0] #store voltage trace of step i
    start = len(mon.Iclamp[0]) #determine start of voltage trace of step i
    k = k+1 #update step count

Voltage clamp Sawtooth Mouse L2/3 Barrel cortex Pyramidal Neuron Model.

In [5]:
#passive neuronal parameters
Cm = 0.75*uF/cm**2 #membrane Capicitance
Ri = 200*ohm*cm #axial Resistance
EL = -70*mV #leak reversal potential
ENa = 55*mV #sodium reversal potential
EK = -90*mV #potassium reversal potential

#additional parameters
defaultclock.dt = 0.025*ms
duration = 500*ms
param = math.log(10) # used in neuron equations

#neuron Morphology
morpho = Soma(50*um)  #create soma
diameters = hstack([linspace(4, 1, 11), ones(290)])*um #create tapering
morpho.axon = Section(diameter=diameters, length=ones(300)*um, n=300) # Create Axon

#set neuron equations
eqs='''
Im = gL*(EL - v) + (m**3)*gNa*h*(ENa - v) + (n**4)*gK*(EK - v) + gClamp*(vc-v) : amp/meter**2
Iclamp = gClamp*(vc-v) : amp/meter**2

# activation gates Na channel
dm/dt = (minf-m)/taum : 1 
minf = (1 / (1 + exp((v+25.5*mV) / (-5.29*mV)))) : 1
taum = 2.64*ms - (2.52*ms/(1 + exp((v+120*mV) / (-25*mV)))) : second

# inactivation gate Na channel
dh/dt = (hinf-h)/tauh : 1
hinf = (1 / (1 + exp((v+48.9*mV) / (5.18*mV)))) : 1
tauh = (2*ms/(1 + exp((v+62.9*mV) / (-10*mV)))) + (1*ms/(1 + exp((v+34.9*mV) / (3.6*mV)))) : second

# activation gates K channel
dn/dt = alphan * (1-n) - betan * n : 1
alphan =(1 / exp(((param*((-0.021 * v + (1.14*mV))/mV)))))/ms : Hz
betan = (1 / exp(((param*((0.031 * v + (2.8*mV))/mV)))))/ms : Hz

gNa : siemens/metre**2
gK : siemens/metre**2
gL : siemens/metre**2
gClamp : siemens/metre**2

vc = EL * ((4/p) * (abs((t % p) - (p/2)) - (p/4))) : volt (shared)  #sawtooth with period p
'''

#create spatial neuron
neuron = SpatialNeuron(morphology=morpho, model=eqs, Cm=Cm, Ri=Ri, threshold='v > -40*mV',
                    refractory='v > -40*mV', method='exponential_euler')

#set neuronal parameters
neuron.gL[0] = 0.5*nS/morpho.area[0] #leak conductance soma
neuron.axon.gL = 0.33*nS/sum(morpho.axon.area) #leak conductance whole axon incl. AIS

neuron.gK[0] = 10*nS/morpho.area[0] #K conductance soma
neuron.axon.gK[10:25] = 10*nS/sum(morpho.axon.area[10:25]) # K conductance AIS

neuron.gNa[0] = 300*nS/morpho.area[0] #NA conductance soma
neuron.axon[10:25].gNa = 3000*nS/sum(morpho.axon.area[10:25]) # NA conductance AIS

neuron.v = EL #membrane voltage
neuron.h = 1 #inactivation gate NA
neuron.m = 0 #activation gate NA
neuron.n = 0.5 #activation gate K

neuron.gClamp = (0.5*nS/morpho.area[0])*100 #injected current conductance

#set sawtooth period (for one round = up and down)
p = 200*ms

#monitor required variables
spikes = SpikeMonitor(neuron) # record spikes
mon = StateMonitor(neuron, ['Iclamp'], record = True) # record additional neuron parameters

#run
run(duration)